In [ ]:
!cd /content    # switch  to /content directory

In [ ]:
from google.colab import files    # for downloading dataset from kaggle

In [ ]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"brahmdevpandey71","key":"4b92372a3b72bdce770ab4eceb0e1af7"}'}

In [ ]:
!mkdir ~/.kaggle

In [ ]:
!mv ./kaggle.json ~/.kaggle

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!kaggle datasets download -d kazanova/sentiment140

 80% 65.0M/80.9M [00:00<00:00, 164MB/s]
100% 80.9M/80.9M [00:00<00:00, 166MB/s]


In [ ]:
!unzip sen*

Archive:  sentiment140.zip
  inflating: training.1600000.processed.noemoticon.csv  


In [ ]:
!pip install contractions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 kB 10.9 MB/s eta 0:00:00


In [ ]:
!pip install spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd   # to manage data of csv
from concurrent.futures import ThreadPoolExecutor   # for parallel processing
from multiprocessing import cpu_count   # to get the number of threads available in the threads
import contractions   # for performing contractions expansion on text
import re   # for regular expressions to remove useless symbols from the text
from nltk import word_tokenize
import nltk
from spacy.lang.en import stop_words
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from nltk.tag import pos_tag
from nltk.stem import WordNetLemmatizer

/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
nltk.download("wordnet")

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
stopwords = stopwords.words("english")
STOP_WORDS = stop_words.STOP_WORDS
en_stop_words = set.union(set(stopwords), set(STOP_WORDS))

In [ ]:
cpu_count()

2

In [ ]:
csv_path = "/content/training.1600000.processed.noemoticon.csv"
data = pd.read_csv(csv_path, encoding="latin-1", header=None)
data

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [ ]:
data.shape
print(set(data[0]))

{0, 4}


In [ ]:
data.rename(columns={0: "target", 1: "ids", 2: "date", 3: "flag", 4: "user", 5: "text"}, inplace=True)
data

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [ ]:
data.drop(columns=["ids", "date", "flag", "user"], inplace=True)
data

,target,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."
...,...,...
1599995,4,Just woke up. Having no school is the best fee...
1599996,4,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,Happy 38th Birthday to my boo of alll time!!! ...


In [ ]:
print(set(data[data.columns[0]]))

{0, 4}


In [ ]:
# function to normalize tweets text
def normalize_tweet(tweet_text):
  return tweet_text.lower()

In [ ]:
# normalize the text of the tweets
with ThreadPoolExecutor(max_workers=4) as pool:
  data["text"] = list(pool.map(nor  malize_tweet, list(data["text"])))

In [ ]:
data

,target,text
0,0,"@switchfoot http://twitpic.com/2y1zl - awww, t..."
1,0,is upset that he can't update his facebook by ...
2,0,@kenichan i dived many times for the ball. man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."
...,...,...
1599995,4,just woke up. having no school is the best fee...
1599996,4,thewdb.com - very cool to hear old walt interv...
1599997,4,are you ready for your mojo makeover? ask me f...
1599998,4,happy 38th birthday to my boo of alll time!!! ...


In [ ]:
regex_pattern = r'^@[a-zA-z0-9 ]+|^#[a-zA-Z0-9 ]+|\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*|\W+|\d+|<("[^"]*"|\'[^\']*\'|[^\'">])*>|_+|[^\u0000-\u007f]+'

In [ ]:
def remove_noisy_tokens(tweet_text):
  return re.sub(pattern=regex_pattern, repl=" ", string=tweet_text)

In [ ]:
with ThreadPoolExecutor(max_workers=4) as pool:
  data["text"] = list(pool.map(remove_noisy_tokens, list(data["text"])))

In [ ]:
data["text"]

0            twitpic com  y zl awww that s a bummer you s...
1          is upset that he can t update his facebook by ...
2                managed to save   the rest go out of bounds
3            my whole body feels itchy and like its on fire 
4            it s not behaving at all i m mad why am i he...
                                 ...                        
1599995    just woke up having no school is the best feel...
1599996    thewdb com very cool to hear old walt intervie...
1599997    are you ready for your mojo makeover ask me fo...
1599998    happy  th birthday to my boo of alll time tupa...
1599999    happy charitytuesday thenspcc sparkscharity sp...
Name: text, Length: 1600000, dtype: object

In [ ]:
def remove_remaining_noisy_tokens(tweet_text):
  return re.sub(r'\b\w\b|[^\u0000-\u007f]+|_+|\W+', repl=" ", string=tweet_text)

In [ ]:
with ThreadPoolExecutor(max_workers=4) as pool:
  data["text"] = list(pool.map(remove_remaining_noisy_tokens, list(data["text"])))

In [ ]:
data["text"].head()

0     twitpic com   zl awww that     bummer you sho...
1    is upset that he can   update his facebook by ...
2            managed to save the rest go out of bounds
3      my whole body feels itchy and like its on fire 
4     it   not behaving at all     mad why am   her...
Name: text, dtype: object

In [ ]:
def tokenize_tweet_text(tweet_text):
  return word_tokenize(tweet_text)

In [ ]:
with ThreadPoolExecutor(max_workers=4) as pool:
  data["text"] = list(pool.map(tokenize_tweet_text, list(data["text"])))

In [ ]:
def is_stopword(token_text):
  return not(token_text in en_stop_words)

In [ ]:
def remove_stopwords(tweet_tokens):
  return list(filter(is_stopword, tweet_tokens))

In [ ]:
with ThreadPoolExecutor(max_workers=4) as pool:
  data["text"] = list(pool.map(remove_stopwords, list(data["text"])))

In [ ]:
data["text"].head()

0    [twitpic, com, zl, awww, bummer, shoulda, got,...
1    [upset, update, facebook, texting, cry, result...
2                        [managed, save, rest, bounds]
3                     [body, feels, itchy, like, fire]
4                                      [behaving, mad]
Name: text, dtype: object

In [ ]:
def translate_pos_tag(token_treebank_tag):
  if token_treebank_tag[1].startswith('J'):
    return (token_treebank_tag[0], wn.ADJ)
  elif token_treebank_tag[1].startswith('V'):
    return (token_treebank_tag[0], wn.VERB)
  elif token_treebank_tag[1].startswith('N'):
    return (token_treebank_tag[0], wn.NOUN)
  elif token_treebank_tag[1].startswith('R'):
    return (token_treebank_tag[0], wn.ADV)
  else:
    return (token_treebank_tag[0], wn.NOUN)

In [ ]:
def tag_token(tweet_tokens):
  return list(map(translate_pos_tag, pos_tag(tweet_tokens)))

In [ ]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
data["text"] = list(map(tag_token, list(data["text"])))

In [ ]:
data["text"].head()

0    [(twitpic, n), (com, n), (zl, n), (awww, n), (...
1    [(upset, a), (update, a), (facebook, n), (text...
2    [(managed, v), (save, v), (rest, n), (bounds, n)]
3    [(body, n), (feels, n), (itchy, v), (like, n),...
4                            [(behaving, v), (mad, n)]
Name: text, dtype: object

In [ ]:
lemmatizer = WordNetLemmatizer()  # create an instance of wordnet lemmatizer

In [ ]:
# function to lemmatize each token of the list
def lemmatize_token(token_pos_tuple):
  if token_pos_tuple == None:
    return ""
  else:
    return lemmatizer.lemmatize(word=token_pos_tuple[0], pos=token_pos_tuple[1])

In [ ]:
def lemmatize_tweet_tokens(tweet_tokens):
  if len(tweet_tokens) > 0:
    return list(map(lemmatize_token, tweet_tokens))
  else:
    return [""]

In [ ]:
data["text"] = list(map(lemmatize_tweet_tokens, list(data["text"])))

In [ ]:
data["text"].head()

0    [twitpic, com, zl, awww, bummer, shoulda, get,...
1    [upset, update, facebook, texting, cry, result...
2                          [manage, save, rest, bound]
3                      [body, feel, itchy, like, fire]
4                                        [behave, mad]
Name: text, dtype: object

In [ ]:
def join_tokens(tweet_tokens):
  return " ".join(tweet_tokens)

In [ ]:
with ThreadPoolExecutor(max_workers=4) as pool:
  data["text"] = list(pool.map(join_tokens, list(data["text"])))

In [ ]:
data["text"].head()

0    twitpic com zl awww bummer shoulda get david c...
1    upset update facebook texting cry result schoo...
2                               manage save rest bound
3                            body feel itchy like fire
4                                           behave mad
Name: text, dtype: object

In [ ]:
data

,target,text
0,0,twitpic com zl awww bummer shoulda get david c...
1,0,upset update facebook texting cry result schoo...
2,0,manage save rest bound
3,0,body feel itchy like fire
4,0,behave mad
...,...,...
1599995,4,woke school best feeling
1599996,4,thewdb com cool hear old walt interview
1599997,4,ready mojo makeover ask detail
1599998,4,happy th birthday boo alll time tupac amaru sh...


In [ ]:
data.to_csv('/content/drive/MyDrive/sentiment analysis files/tokenized_data_joined.csv')